# Runtime of `RegGridProlongator` and `RegGridInterpolator`
### ( `emg3d.solver.RegGridProlongator` and `scipy.interpolate.RegGridInterpolator`)

For background and motivation see: https://mail.python.org/pipermail/scipy-dev/2019-May/023537.html.

Requires [emg3d](https://empymod.github.io) v0.6.0 or higher:
```
conda install -c prisae emg3d
```
or
```
pip install emg3d
```

In [1]:
import numpy as np
import scipy.interpolate as si
from emg3d import utils, solver

In [2]:
def prolon_scipy(grid, cgrid, efield, cefield, x_points):
    """Calculate SciPy alternative."""
    
    for ixc in range(cgrid.nCx):
        
        # Bilinear interpolation in the y-z plane
        fn = si.RegularGridInterpolator(
                (cgrid.vectorNy, cgrid.vectorNz), cefield.fx[ixc, :, :],
                bounds_error=False, fill_value=None)
        hh = fn(x_points).reshape(grid.vnEx[1:], order='F')

        # Piecewise constant interpolation in x-direction
        efield[2*ixc, :, :] += hh
        efield[2*ixc+1, :, :] += hh
        
    return efield
        

def prolon_emg3d(grid, cgrid, efield, cefield, x_points):
    """Calculate emg3d alternative."""
    
    fn = solver.RegularGridProlongator(cgrid.vectorNy, cgrid.vectorNz, x_points)
    for ixc in range(cgrid.nCx):
        
        # Bilinear interpolation in the y-z plane
        hh = fn(cefield.fx[ixc, :, :]).reshape(grid.vnEx[1:], order='F')
        
        # Piecewise constant interpolation in x-direction
        efield[2*ixc, :, :] += hh
        efield[2*ixc+1, :, :] += hh
        
    return efield

In [3]:
def test_prolong(n):
    # Nr of cells of fine grid.
    nx = 2**n

    # Create fine grid.
    hx = 50*np.ones(nx)
    grid = utils.TensorMesh([hx, hx, hx], x0=np.array([0, 0, 0]))

    # Create coarse grid.
    chx = np.diff(grid.vectorNx[::2])
    cgrid = utils.TensorMesh([chx, chx, chx], x0=np.array([0, 0, 0]))
    print(f"\n  === n : {n} ===; fine: {grid.nC}, coarse: {cgrid.nC} (first SciPy, then emg3d)")

    # Create empty fine grid fields.
    efield1 = utils.Field(grid)
    efield2 = utils.Field(grid)

    # Create coarse grid field with some values.
    cefield = utils.Field(cgrid)
    cefield.fx = np.arange(cefield.fx.size)
    cefield.fx = 1j*np.arange(cefield.fx.size)/10

    # Required interpolation points.
    yz_points = solver._get_prolongation_coordinates(grid, 'y', 'z')

    # Timeit
    t1 = %timeit -o prolon_scipy(grid, cgrid, efield1.fx.copy(), cefield, yz_points)
    t2 = %timeit -o prolon_emg3d(grid, cgrid, efield2.fx.copy(), cefield, yz_points)
    print(f"Speedup: {t1.best/t2.best:.2f}")
    
    # Compare
    out1 = prolon_scipy(grid, cgrid, efield1.fx.copy(), cefield, yz_points)
    out2 = prolon_emg3d(grid, cgrid, efield2.fx.copy(), cefield, yz_points)
    np.allclose(out1, out2, rtol=1e-7, atol=0)  # Same as assert_allclose

In [4]:
for n in range(2, 9):
    test_prolong(n)


  === n : 2 ===; fine: 64, coarse: 8 (first SciPy, then emg3d)
364 µs ± 22.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
192 µs ± 36.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Speedup: 2.14

  === n : 3 ===; fine: 512, coarse: 64 (first SciPy, then emg3d)
709 µs ± 18.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
289 µs ± 9.67 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Speedup: 2.44

  === n : 4 ===; fine: 4096, coarse: 512 (first SciPy, then emg3d)
1.69 ms ± 30.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
578 µs ± 2.77 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Speedup: 2.85

  === n : 5 ===; fine: 32768, coarse: 4096 (first SciPy, then emg3d)
4.91 ms ± 212 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.62 ms ± 29.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Speedup: 2.99

  === n : 6 ===; fine: 262144, coarse: 32768 (first SciPy, then emg3d)
19.9 ms ± 735 µs per loop 

In [5]:
utils.Versions()

------------------------------------------------------
  Tue May 28 07:03:55 2019 CEST

          Linux : OS
              4 : CPU(s)
         1.16.2 : numpy
          1.2.1 : scipy
         0.43.1 : numba
          0.6.0 : emg3d
          7.4.0 : IPython
          3.0.3 : matplotlib

  3.7.3 (default, Mar 27 2019, 22:11:17)  [GCC
  7.3.0]

  Intel(R) Math Kernel Library Version 2019.0.3
  Product Build 20190125 for Intel(R) 64
  architecture applications
------------------------------------------------------